In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
ais_dataset = pd.read_csv('./datasets/ais_dataset.csv')
distance_dataset = pd.read_csv('./datasets/distance_dataset.csv')
port_call_dataset = pd.read_csv('./datasets/port_call_dataset.csv')

print(ais_dataset.info())
# no terminal, maneuvering , berth
# suggests that only anchored and transit
# we can simply find if anchored or not

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251769 entries, 0 to 251768
Data columns (total 30 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   imo                    251769 non-null  int64  
 1   mmsi                   251769 non-null  int64  
 2   vessel_name            251769 non-null  object 
 3   date_of_build          251769 non-null  object 
 4   vessel_type            251769 non-null  object 
 5   group                  251769 non-null  object 
 6   timestamp              251769 non-null  object 
 7   lon                    251769 non-null  float64
 8   lat                    251769 non-null  float64
 9   nav_stat               251769 non-null  int64  
 10  speed                  249752 non-null  float64
 11  course                 251769 non-null  int64  
 12  heading                250464 non-null  float64
 13  fuel_category          251769 non-null  int64  
 14  main_engine_fuel_type  251769 non-nu

2543